In [2]:
import torch
from torch import nn
from torch.nn import functional as F

In [3]:
net = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
X = torch.rand(2, 20)

In [4]:
net(X)

tensor([[ 0.2685, -0.0645, -0.0248, -0.0268, -0.1197,  0.2688,  0.0534,  0.0428,
         -0.0588, -0.1409],
        [ 0.1979, -0.0157, -0.0587,  0.0471, -0.1980,  0.2599,  0.0746, -0.0175,
         -0.0961, -0.1581]], grad_fn=<AddmmBackward0>)

In [5]:
# 自定义块
class MLP(nn.Module):
    def __init__(self):
        super().__init__()  # 调用父类的init函数
        self.hidden = nn.Linear(20, 256)
        self.out = nn.Linear(256, 10)

    def forward(self, X):
        return self.out(F.relu(self.hidden(X)))


# 实例化多层感知机的层，然后在每次调用正向传播函数调用这些层
net = MLP()
X = torch.rand(2, 20)
net(X)

tensor([[-0.0765, -0.0338, -0.1119,  0.0367, -0.0188, -0.1994,  0.0456,  0.1888,
         -0.1979, -0.2486],
        [ 0.0696, -0.0305,  0.1507, -0.1141,  0.1788, -0.1051, -0.0273,  0.2049,
         -0.1853, -0.1317]], grad_fn=<AddmmBackward0>)

In [6]:
# 顺序块
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for block in args:
            self._modules[block] = block  # block 本身作为它的key，存在_modules里面的为层，以字典的形式

    def forward(self, X):
        for block in self._modules.values():
            print(block)
            X = block(X)
        return X


net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
X = torch.rand(2, 20)
net(X)

Linear(in_features=20, out_features=256, bias=True)
ReLU()
Linear(in_features=256, out_features=10, bias=True)


tensor([[-0.1236, -0.0175,  0.2058, -0.0042, -0.0282,  0.0230,  0.1774, -0.0791,
          0.1300,  0.0483],
        [-0.1713, -0.0826,  0.3385, -0.1329, -0.1730, -0.0479,  0.1271, -0.1259,
          0.2238,  0.1106]], grad_fn=<AddmmBackward0>)

In [10]:
# 正向传播
# 在正向传播函数中执行代码
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.rand_weight = torch.rand((20, 20), requires_grad=True)
        self.linear = nn.Linear(20, 20)

    def forward(self, X):
        X = self.linear(X)
        X = F.relu(torch.mm(X, self.rand_weight + 1))
        X = self.linear(X)
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()


net = FixedHiddenMLP()
X = torch.rand(2, 20)
net(X)

tensor(0.1004, grad_fn=<SumBackward0>)

In [11]:
# 混合组合块
# 混合代培各种组合块的方法
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(),
                                 nn.Linear(64, 32), nn.ReLU())
        self.linear = nn.Linear(32, 16)

    def forward(self, X):
        return self.linear(self.net(X))


chimear = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
X = torch.rand(2, 20)
chimear(X)

tensor(-0.0981, grad_fn=<SumBackward0>)

In [13]:
# 参数管理
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 1))
X = torch.rand(size=(2, 4))
print(net(X))

tensor([[-0.5491],
        [-0.5629]], grad_fn=<AddmmBackward0>)


In [14]:
print(net[2].state_dict())  # 访问参数，net[2]就是最后一个输出层

OrderedDict([('weight', tensor([[-0.1544, -0.2595,  0.2291,  0.2240, -0.2738,  0.3294, -0.3034,  0.1729]])), ('bias', tensor([-0.3269]))])


In [15]:
print(type(net[2].bias))  # 目标参数

<class 'torch.nn.parameter.Parameter'>


In [16]:
print(net[2].bias)

Parameter containing:
tensor([-0.3269], requires_grad=True)


In [17]:
print(net[2].bias.data)

tensor([-0.3269])


In [18]:
print(net[2].weight.grad == None)  # 还没进行反向计算，所以grad为None

True


In [19]:
print(*[(name, param.shape) for name, param in net[0].named_parameters()])  # 一次性访问所有参数

('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))


In [20]:
print(*[(name, param.shape) for name, param in net.named_parameters()])  # 0是第一层名字，1是ReLU，它没有参数

('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))


In [21]:
print(net.state_dict()['2.bias'].data)  # 通过名字获取参数

tensor([-0.3269])


In [22]:
# 嵌套块
# 从嵌套块收集参数
def block1():
    return nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 4), nn.ReLU())


def block2():
    net = nn.Sequential()
    for i in range(4):
        net.add_module(f'block{i}', block1())  # f'block{i}' 可以传一个字符串名字过来，block2可以嵌套四个block1
    return net


rgnet = nn.Sequential(block2(), nn.Linear(4, 1))
print(rgnet(X))
print(rgnet)

tensor([[-0.2465],
        [-0.2465]], grad_fn=<AddmmBackward0>)
Sequential(
  (0): Sequential(
    (block0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)


In [23]:
# 内置初始化
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 1))


def init_normal(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, mean=0, std=0.01)  # 下划线表示把m.weight的值替换掉
        nn.init.zeros_(m.bias)


net.apply(init_normal)  # 会递归调用 直到所有层都初始化
print(net[0].weight.data[0])
print(net[0].bias.data[0])

tensor([0.0185, 0.0119, 0.0140, 0.0107])
tensor(0.)


In [24]:
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 1))


def init_constant(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 1)  # 使用第二个参数的值填充第一个参数
        nn.init.zeros_(m.bias)


net.apply(init_constant)
print(net[0].weight.data[0])
print(net[0].bias.data[0])

tensor([1., 1., 1., 1.])
tensor(0.)


In [25]:
# 对某些块应用不同的初始化
def xavier(m):
    if type(m) == nn.Linear:
        nn.init.xavier_uniform_(m.weight)  #使用均匀分布 用值填充输入张量


def init_42(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 42)


net[0].apply(xavier)
net[2].apply(init_42)
print(net[0].weight.data[0])
print(net[2].weight.data)

tensor([-0.6328,  0.5538, -0.5029,  0.2752])
tensor([[42., 42., 42., 42., 42., 42., 42., 42.]])


In [26]:
# 参数替换
# 自定义初始化
def my_init(m):
    if type(m) == nn.Linear:
        print("Init", *[(name, param.shape) for name, param in m.named_parameters()][0])  # 打印名字是啥，形状是啥
        nn.init.uniform_(m.weight, -10, 10)
        m.weight.data *= m.weight.data.abs() >= 5  # 这里*=的代码相当于先计算一个布尔矩阵(先判断>=)，然后再用布尔矩阵的对应元素去乘以原始矩阵的每个元素。保留绝对值大于5的权重，不是的话就设为0


net.apply(my_init)
print(net[0].weight[:2])
net[0].weight.data[:] += 1  # 参数替换
net[0].weight.data[0, 0] = 42
print(net[0].weight.data[0])

Init weight torch.Size([8, 4])
Init weight torch.Size([1, 8])
tensor([[ 0.0000, -5.6865, -0.0000,  0.0000],
        [-9.5521,  6.4465,  0.0000,  0.0000]], grad_fn=<SliceBackward0>)
tensor([42.0000, -4.6865,  1.0000,  1.0000])


In [27]:
# 参数绑定
shared = nn.Linear(8,8)
net = nn.Sequential(nn.Linear(4,8),nn.ReLU(),shared,nn.ReLU(),shared,nn.ReLU(),nn.Linear(8,1))  # 第2个隐藏层和第3个隐藏层是share权重的，第一个和第四个是自己的
net(X)
print(net[2].weight.data[0] == net[4].weight.data[0])
net[2].weight.data[0,0] = 100
print(net[2].weight.data[0] == net[4].weight.data[0])

tensor([True, True, True, True, True, True, True, True])
tensor([True, True, True, True, True, True, True, True])


In [28]:
# 构造一个没有任何参数的自定义层
class CenteredLayer(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, X):
        return X - X.mean()

In [29]:
layer = CenteredLayer()
print(layer(torch.FloatTensor([1,2,3,4,5])))

tensor([-2., -1.,  0.,  1.,  2.])


In [30]:
# 将层作为组件合并到构建更复杂的模型中
net = nn.Sequential(nn.Linear(8,128),CenteredLayer())
Y = net(torch.rand(4,8))
print(Y.mean())

tensor(6.5193e-09, grad_fn=<MeanBackward0>)


In [31]:
# 带参数的图层
class MyLinear(nn.Module):
    def __init__(self, in_units, units):
        super().__init__()
        self.weight = nn.Parameter(torch.randn(in_units,units)) # nn.Parameter使得这些参数加上了梯度
        self.bias = nn.Parameter(torch.randn(units,))

    def forward(self, X):
        linear = torch.matmul(X, self.weight.data) + self.bias.data
        return F.relu(linear)

In [32]:
dense = MyLinear(5,3)
print(dense.weight)

Parameter containing:
tensor([[ 3.1311e-01,  9.2675e-01, -6.0626e-01],
        [-1.1431e-01,  1.2902e+00, -3.2219e-01],
        [-2.7518e-01, -7.4910e-01, -1.1032e+00],
        [ 2.2255e-01, -1.6355e+00, -4.6399e-05],
        [ 2.7456e-02,  3.2571e+00, -4.9720e-01]], requires_grad=True)


In [33]:
# 使用自定义层直接执行正向传播计算
print(dense(torch.rand(2,5)))

tensor([[0.0000, 1.2643, 0.0000],
        [0.0000, 1.9361, 0.0000]])


In [34]:
# 使用自定义层构建模型
net = nn.Sequential(MyLinear(64,8),MyLinear(8,1))
print(net(torch.rand(2,64)))

tensor([[0.],
        [0.]])


In [37]:
x = torch.arange(4)
torch.save(x, './data/x-file')

In [38]:
x2 = torch.load("./data/x-file")
print(x2)

tensor([0, 1, 2, 3])


In [39]:
#存储一个张量列表，然后把它们读回内存
y = torch.zeros(4)
torch.save([x,y],'./data/x-files')

In [42]:
x2, y2 = torch.load('./data/x-files')
print(x2)
print(y2)

tensor([0, 1, 2, 3])
tensor([0., 0., 0., 0.])


In [43]:
# 写入或读取从字符串映射到张量的字典
mydict = {'x':x,'y':y}
torch.save(mydict,'./data/mydict')
mydict2 = torch.load('./data/mydict')
print(mydict2)

{'x': tensor([0, 1, 2, 3]), 'y': tensor([0., 0., 0., 0.])}


In [44]:
# 加载和保存模型参数
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20,256)
        self.output = nn.Linear(256,10)

    def forward(self, x):
        return self.output(F.relu(self.hidden(x)))

In [45]:
net = MLP()
X = torch.randn(size=(2,20))
Y = net(X)

In [46]:
# 将模型的参数存储为一个叫做"mlp.params"的文件
torch.save(net.state_dict(),'./data/mlp.params')

In [47]:
# 实例化了原始多层感知机模型的一个备份。直接读取文件中存储的参数
clone = MLP() # 必须要先声明一下，才能导入参数
clone.load_state_dict(torch.load("./data/mlp.params"))
print(clone.eval()) # eval()是进入测试模式

MLP(
  (hidden): Linear(in_features=20, out_features=256, bias=True)
  (output): Linear(in_features=256, out_features=10, bias=True)
)


In [48]:
Y_clone = clone(X)
print(Y_clone == Y)

tensor([[True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True]])
